# Preguntas

- Cual es el numero de tarea? The doc says proyecto 1

Proyecto 1 - Luis Eduardo Robles Jimenez

# Minería de Texto para Turismo

## Utils

In [1]:
import os
import pandas as pd

In [2]:
class TouristSpot:
    def __init__(self, path):
        self.reviews = pd.read_csv(path)
        self._cleanUp()

    def _cleanUp(self):
        columns = ["Título de la opinión", "Opinión"]
        for col in columns:
            desc = []       
            for o in self.reviews.loc[:, col]:
                if o[0] == '"': o = o[1:]
                if o[-1] == '"': o = o[:-1]
                desc.append(o)
            self.reviews.loc[:, col] = desc

    def describe():
        pass

## Actividades

### 1. Preprocesamiento y stats

In [3]:
path = "../../data/guanajuato/"

In [4]:
for d in os.listdir(path):
    TouristSpot(os.path.join(path, d))
    break